In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
import torch
import traceback

# Path to your saved model
model_path = "./unbiased_summarizer_dpo_final_1000"

print(f"Checking model at: {model_path}")

try:
    # Determine device
    device = "cuda" if torch.cuda.is_available() else "cpu"
    print(f"Using device: {device}")

    # Load Tokenizer
    print("Loading tokenizer...")
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    print(f"Special Tokens: BOS={tokenizer.bos_token_id}, EOS={tokenizer.eos_token_id}, PAD={tokenizer.pad_token_id}")
    
    # Load Model
    print("Loading model...")
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
    print("Model loaded successfully!")
    
    # --- FIX CONFIGURATION ISSUES GLOBALLY ---
    # Explicitly set config values to prevent "NoneType" and "early_stopping" errors
    model.config.early_stopping = True
    model.config.forced_bos_token_id = 0
    if model.config.decoder_start_token_id is None:
        model.config.decoder_start_token_id = model.config.eos_token_id
    
    # Check for NaN weights (sign of corrupted training)
    print("Checking model weights...")
    for name, param in list(model.named_parameters())[:5]:
        if torch.isnan(param).any():
            print(f"WARNING: Parameter {name} contains NaNs!")
        if param.sum() == 0:
             print(f"WARNING: Parameter {name} is all zeros!")

    # Test Generation
    sample_text = """
    Artificial intelligence (AI) is intelligence demonstrated by machines, as opposed to the natural intelligence displayed by animals including humans. AI research has been defined as the field of study of intelligent agents, which refers to any system that perceives its environment and takes actions that maximize its chance of achieving its goals.
    The term "artificial intelligence" had previously been used to describe machines that mimic and display "human" cognitive skills that are associated with the human mind, such as "learning" and "problem-solving". This definition has since been rejected by major AI researchers who now describe AI in terms of rationality and acting rationally, which does not limit how intelligence can be articulated.
    """
    
    print("\nGenerating test summary...")
    inputs = tokenizer(sample_text, return_tensors="pt", max_length=1024, truncation=True).to(device)
    
    # Debug: Try generation with minimal constraints
    print("\n--- Attempt 1: Greedy Search (No Forcing) ---")
    summary_ids = model.generate(
        inputs["input_ids"], 
        max_length=100, 
        min_length=10, 
        do_sample=False, 
        num_beams=1
    )
    print(f"IDs: {summary_ids[0].tolist()}")
    print(f"Text: {tokenizer.decode(summary_ids[0], skip_special_tokens=True)}")

    # Debug: Try generation with Sampling + No Repeat N-Grams (PPO Style)
    print("\n--- Attempt 2: Sampling with Constraints (PPO Style) ---")
    summary_ids = model.generate(
        inputs["input_ids"], 
        max_length=100, 
        min_length=30, 
        do_sample=True,              # Enable Sampling
        num_beams=1,                 # Explicitly set to 1 for sampling
        top_p=0.9,                   # Nucleus sampling
        no_repeat_ngram_size=3,      # Prevent repetition
        early_stopping=False         # Disable early_stopping for sampling
    )
    print(f"IDs: {summary_ids[0].tolist()}")
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    
    print("-" * 30)
    print("Generated Summary:")
    print(summary)
    print("-" * 30)
    
    if not summary.strip():
        print("WARNING: Summary is empty. The model might be generating only special tokens.")
    else:
        print("Verification Complete: Model is functional.")

except Exception as e:
    print("\n!!! Model Verification Failed !!!")
    print(f"Error: {e}")
    traceback.print_exc()

/home/tmittal/CSE595/.arc_env/lib64/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Checking model at: ./unbiased_summarizer_dpo_final_1000
Using device: cuda
Loading tokenizer...
Special Tokens: BOS=0, EOS=2, PAD=1
Loading model...


/home/tmittal/CSE595/.arc_env/lib64/python3.12/site-packages/transformers/models/bart/configuration_bart.py:177: UserWarning: Please make sure the config includes `forced_bos_token_id=0` in future versions. The config can simply be saved and uploaded again to be fixed.
  warnings.warn(



!!! Model Verification Failed !!!
Error: CUDA error: CUDA-capable device(s) is/are busy or unavailable
Search for `cudaErrorDevicesUnavailable' in https://docs.nvidia.com/cuda/cuda-runtime-api/group__CUDART__TYPES.html for more information.
CUDA kernel errors might be asynchronously reported at some other API call, so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1
Compile with `TORCH_USE_CUDA_DSA` to enable device-side assertions.



Traceback (most recent call last):
  File "/tmp/ipykernel_3633966/3012275925.py", line 22, in <module>
    model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/tmittal/CSE595/.arc_env/lib64/python3.12/site-packages/transformers/modeling_utils.py", line 4343, in to
    return super().to(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/tmittal/CSE595/.arc_env/lib64/python3.12/site-packages/torch/nn/modules/module.py", line 1371, in to
    return self._apply(convert)
           ^^^^^^^^^^^^^^^^^^^^
  File "/home/tmittal/CSE595/.arc_env/lib64/python3.12/site-packages/torch/nn/modules/module.py", line 930, in _apply
    module._apply(fn)
  File "/home/tmittal/CSE595/.arc_env/lib64/python3.12/site-packages/torch/nn/modules/module.py", line 930, in _apply
    module._apply(fn)
  File "/home/tmittal/CSE595/.arc_env/lib64/python3.12/site-packages/torch/nn/modules/module.py